<a href="https://colab.research.google.com/github/travelingshoez/amazon_vine_analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,544 kB]
Hit:14 http://ppa.launchpad.net/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-14 18:28:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.92MB/s    in 0.2s    

2023-03-14 18:28:55 (4.92 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

### Create DataFrames to match tables

In [5]:
# Read in the Review dataset as a DataFrame
from pyspark import SparkFiles
from pyspark.sql.functions import to_date
review_df = df.select('review_id','review_headline', 'review_body', 'review_date')
review_df.show(20)

+--------------+--------------------+--------------------+-----------+
|     review_id|     review_headline|         review_body|review_date|
+--------------+--------------------+--------------------+-----------+
|R1WBPB8MDCCN8F|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|R32M0YEWV77XG8|       Shipped fast.|These are the bes...| 2015-08-31|
| RR8V7WR27NXJ5|Good idea if it w...|It looks good, th...| 2015-08-31|
|R1MHO5V9Z932AY|          Five Stars|           Great jig| 2015-08-31|
|R16PD71086BD2V|Great quality sil...|I love swimming i...| 2015-08-31|
|R1Z8IFGWTRWXT6|Love everything a...|Love everything a...| 2015-08-31|
|R3AUMSHAW73HWN|          Four Stars|not the best sock...| 2015-08-31|
|R2KWDWFOHGX6FL|          Five Stars|             Love it| 2015-08-31|
|R3H9543FWBWFBU|can't wait to use it|Unfortunately I h...| 2015-08-31|
| RUANXOQ9W3OU5|great product, ac...|fast shipping , g...| 2015-08-31|
|R31673RTGEZSW7|this mat is a pre...|If you have a ter...| 2015-08-31|
|R22OQ

In [6]:
# Create the customers_table DataFrame

from pyspark.sql.functions import col, count
customers_df = df.groupBy('customer_id').agg(count('customer_id').alias('customer_count'))
customers_df.show(20)




+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35669025|             1|
|   48198949|            30|
|   43355824|             6|
|   33014261|             6|
|   23493243|             1|
|   30717305|             4|
|   15714077|             2|
|    7854719|             1|
|   12761428|             2|
|   14127895|             1|
|   51451778|             4|
|    3452965|             1|
|   40430762|             2|
|   27314089|            15|
|   11405991|             1|
|   20368048|             2|
|   33967841|             2|
|   10418855|             1|
|   43783459|             1|
|   50843047|             4|
+-----------+--------------+
only showing top 20 rows



In [17]:
# Create the products_table DataFrame and drop duplicates. 

from pyspark.sql.functions import countDistinct
products_df.select(countDistinct("product_id")).show()

#products_df = df.select(['product_id','product_title'])
#products_df.nunique()

#products_df['product_id'].nunique()
#products_df.show()

#products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df.show()



+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                   1046150|
+--------------------------+

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B012P7UPSM|Chicago Blackhawk...|
|B001GQ3VHG|Copag Poker Size ...|
|B008VS8M58|Baoer 223 5.56x45...|
|B005F06F4U|All Terrain Tackl...|
|B010T4IE2C|Swim Cap - 3 Pack...|
|B004RKJGLS|adidas Men's Spor...|
|B005V3DCBU|Nike Men's Perfor...|
|B00MHT9WN8|Green Bay Packers...|
|B001CSIRQ8|Isokinetics Inc. ...|
|B001KZ3NOO|Aottop Quality El...|
|B00005RCQS|Everlast 2'x6' Fo...|
|B00FA7RWVI|OGIO Men's Shredd...|
|B000MMH2HU|Blue Sea Systems ...|
|B00WJOATDS|Zengi AR15 45 Deg...|
|B00BOM2JNY|Black Mountain Pr...|
|B00OD5GAFM|Trijicon VCOG 1-6...|
|B00CAHDC1K|Naruto Headband f...|
|B004NLHXLG|Under Armour Men'...|
|B00L7OANWI|Yes4All Deep Tiss...|
|B0085PPSIQ|Lansky PS-MED01 B...|
+----------+--------------------+
only showing top 20 rows



In [19]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
from pyspark.sql.functions import to_date
review_id_df = df.select('review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date', 'yyyy-MM-dd').alias('review_date'))
review_id_df.show(20)





+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

In [20]:
# Create the vine_table. DataFrame
vine_df = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_df.show(20)



+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|
|R32M0YEWV77XG8|          5|            1|          1|   N|
| RR8V7WR27NXJ5|          1|            0|          0|   N|
|R1MHO5V9Z932AY|          5|            0|          0|   N|
|R16PD71086BD2V|          5|            0|          1|   N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|
|R3AUMSHAW73HWN|          4|            2|          3|   N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|
|R3H9543FWBWFBU|          5|            1|          1|   N|
| RUANXOQ9W3OU5|          5|            0|          0|   N|
|R31673RTGEZSW7|          5|            2|          2|   N|
|R22OQLFSH42RCM|          5|            1|          1|   N|
|R12LEL4F3TSZUJ|          5|            2|          2|   N|
|R2L9XWD03072NI|          5|            

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [15]:
# Configure settings for RDS
# amazonvine.crdxx7zzqcz6.us-east-2.rds.amazonaws.com
mode = "append"
jdbc_url="jdbc:postgresql://amazonvine.crdxx7zzqcz6.us-east-2.rds.amazonaws.com:5432/amazon"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"} 
          #restart
          #try test 
          #one db postgres and RDS5432 

          #look for the data review_id
          

In [21]:
# Write review_id_df to table in RDS
#review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


Py4JJavaError: ignored

In [18]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [22]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.3-bin-hadoop2.7/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.2.3-bin-hadoop2.7/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [16]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)